In [53]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [54]:
# Load dataset
df= pd.read_csv('Electric_Vehicle_Population_Datas.csv')
df.head

<bound method NDFrame.head of         VIN (1-10)     County               City State  Postal Code  \
0       5YJ3E1EB0J   Thurston            Olympia    WA      98512.0   
1       WA1AAAGE9M     Kitsap       Port Orchard    WA      98367.0   
2       5YJ3E1EA2J     Yakima             Yakima    WA      98902.0   
3       5YJ3E1EA4N     Yakima             Yakima    WA      98902.0   
4       7SAYGAEE2P  Snohomish            Bothell    WA      98012.0   
...            ...        ...                ...   ...          ...   
200043  JTMAB3FVXR  Snohomish          Snohomish    WA      98290.0   
200044  7FCTGAAA7P     Pierce             Orting    WA      98360.0   
200045  1V2GNPE87P    Spokane            Spokane    WA      99201.0   
200046  1G1RD6E42E  Snohomish  Mountlake Terrace    WA      98043.0   
200047  5YJ3E1EAXP     Kitsap          Bremerton    WA      98311.0   

        Model Year        Make       Model  \
0             2018       TESLA     MODEL 3   
1             2021       

# Check the dataset

In [55]:
df.shape

(200048, 14)

# 1. Check for missing values

## Missing Values: We check for missing values across all columns.

In [56]:
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
VIN (1-10)                 0
County                     4
City                       4
State                      0
Postal Code                4
Model Year                 0
Make                       0
Model                      0
Electric Vehicle Type      0
Electric Range             0
Base MSRP                  0
Legislative District     442
DOL Vehicle ID             0
Electric Utility           4
dtype: int64


# Drop rows with missing values

In [57]:
important_columns = ['County', 'City', 'Postal Code', 'Legislative District', 'Electric Utility']
df = df.dropna(subset=important_columns)

print("\nRows with missing values dropped. New shape:", df.shape)


Rows with missing values dropped. New shape: (199606, 14)


# 2. Remove duplicates (based on VIN)

## Remove Duplicates: We remove any duplicate records based on the VIN, which should uniquely identify each vehicle.

In [58]:
# Count duplicates based on VIN (1-10)
duplicate_vins = df.duplicated(subset=['VIN (1-10)']).sum()
print(f"\nNumber of duplicate VIN entries: {duplicate_vins}")


Number of duplicate VIN entries: 187637


In [59]:
df = df.drop_duplicates(subset=['VIN (1-10)'], keep='first')
print("\nDuplicates removed. New shape:", df.shape)


Duplicates removed. New shape: (11969, 14)


In [17]:
# Number of columns before removing duplicating= 199606
# Number of columns after removing duplicating=11969
199606-11969 

187637

In [ ]:
# Therefore the number of Duplicates are 187637

# 3. Standardize data formats
## Normalize text columns (e.g., Make, Model, Electric Vehicle Type) to lowercase

### Standardize Data: We normalize the text columns by stripping white spaces and converting everything to lowercase to avoid discrepancies. We ensure that Postal Code has 5 digits and that Model Year is an integer.

In [60]:
# Make a copy of the DataFrame to avoid SettingWithCopyWarning
df = df.copy()

# Standardize text data in specific columns (using .loc to avoid the SettingWithCopyWarning)
df.loc[:, text_columns] = df.loc[:, text_columns].apply(lambda x: x.str.strip().str.lower())

In [61]:
# Ensure Postal Code and Model Year have consistent formats
df['Postal Code'] = df['Postal Code'].astype(str).str.zfill(5)  # Ensure 5-digit postal codes
df['Model Year'] = df['Model Year'].astype(int)  # Convert Model Year to integer

# 4. Convert data types for numerical columns

## Convert Data Types: Numerical columns such as Electric Range, Base MSRP, and Model Year are converted to appropriate numeric types.

In [62]:
df['Electric Range'] = pd.to_numeric(df['Electric Range'], errors='coerce')
df['Base MSRP'] = pd.to_numeric(df['Base MSRP'], errors='coerce')
df['Model Year'] = pd.to_numeric(df['Model Year'], errors='coerce')

# 5. Handle outliers
## For Electric Range: Identify unreasonable outliers (e.g., negative values or zeros)
### Handle Outliers: We identify unreasonable outliers (like an electric range of 0 or negative values) and filter them out.

In [63]:
outliers_range = df[(df['Electric Range'] <= 0) | (df['Electric Range'].isnull())]
print("\nOutliers in Electric Range:")
print(outliers_range)


Outliers in Electric Range:
        VIN (1-10)     County        City State Postal Code  Model Year  \
3       5YJ3E1EA4N     yakima      yakima    WA     98902.0        2022   
4       7SAYGAEE2P  snohomish     bothell    WA     98012.0        2023   
8       7FCTGBAA7P     kitsap     poulsbo    WA     98370.0        2023   
16      5YJ3E1EB3N     yakima      yakima    WA     98902.0        2022   
17      WA1LAAGE2P   thurston        yelm    WA     98597.0        2023   
...            ...        ...         ...   ...         ...         ...   
198708  3FMTK1SU4R  snohomish  marysville    WA     98270.0        2024   
198967  3FMTK1SU3R       king    enumclaw    WA     98022.0        2024   
199196  1GYKPPRL8P     yakima      wapato    WA     98951.0        2023   
199219  WA142BFZ0N      mason       allyn    WA     98524.0        2022   
199501  KNDCE3LGXM      lewis    chehalis    WA     98532.0        2021   

            Make           Model           Electric Vehicle Type  \
3 

In [64]:
# Remove rows where Electric Range is <= 0
df = df[df['Electric Range'] > 0]

In [65]:
# Check new shape of the dataset after outlier removal
df.shape

(7071, 14)

# 6. City & County Verification: Check for consistent naming
## Check for unique city and county names
### City & County Verification: We check the uniqueness of city and county names to ensure consistency.

In [66]:
unique_cities = df['City'].unique()
unique_counties = df['County'].unique()
print("\nUnique Cities:")
print(unique_cities)
print("\nUnique Counties:")
print(unique_counties)


Unique Cities:
['olympia' 'port orchard' 'yakima' 'seattle' 'silverdale' 'snohomish'
 'everett' 'coupeville' 'poulsbo' 'greenbank' 'issaquah' 'renton'
 'bainbridge island' 'bothell' 'woodinville' 'lynnwood' 'suquamish'
 'kingston' 'bremerton' 'duvall' 'lake stevens' 'lacey' 'selah' 'wapato'
 'redmond' 'moxee' 'rochester' 'bellevue' 'hansville' 'brier' 'sammamish'
 'mill creek' 'yelm' 'edmonds' 'oak harbor' 'moses lake' 'pullman'
 'mount vernon' 'clinton' 'valley' 'anacortes' 'sedro-woolley'
 'lummi island' 'woodway' 'kirkland' 'normandy park' 'tukwila' 'burien'
 'vancouver' 'kent' 'nine mile falls' 'brush prairie' 'shoreline'
 'ridgefield' 'newcastle' 'kenmore' 'north bend' 'seatac' 'auburn'
 'federal way' 'camas' 'tumwater' 'battle ground' 'washougal'
 'port townsend' 'monroe' 'woodland' 'pacific' 'kelso' 'maple valley'
 'langley' 'white salmon' 'sequim' 'la center' 'port hadlock'
 'lake forest park' 'sunnyside' 'naches' 'covington' 'quilcene' 'longview'
 'medina' 'enumclaw' 'mercer 

## ALL DONE NOW SAVE

In [67]:
# Save the cleaned data to a new CSV
df.to_csv('Cleaned_Electric_Vehicle_Population_Datas.csv', index=False)

print("\nData cleaning complete!")


Data cleaning complete!
